# P1 REST API

- This Jupyter notebook is an example of how to access the RestAPI interface, described at:
  https://doc.particle.one/

# Credentials / Settings

In [1]:
# Make sure you have `pandas` installed.
# If you don't have it installed, install it in your usual way (with pip, conda)

#! pip install requests pandas

import json

import os

import pandas as pd
import requests

In [2]:
# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
#TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:
TOKEN = "e44e7c6b04ef3ea1cfb7a8a67db74751c177259e"

API_URL = "https://data.particle.one"
HEADERS = {"Authorization": f"Token {TOKEN}", "Content-Type": "application/json"}

# Search query structure

Search query is a Python `dict` with the following structure:
```python
query = {
    "text": "",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": []
}
```
The fields are:
- `text`: string. Works as a filter. Free text. Everything that have no match with this phrase will be filtered out.
- `commodity`: list of strings. Works as a filter. You can find valid values in paragraph 7.1 of this notebook.
- `business_category`: string. Works as a filter. You can find valid values in paragraph 7.2 of this notebook.
- `country`: list of strings. Works as a filter. You can find valid values in paragraph 7.3 of this notebook.
- `frequency`: list of strings. Works as a filter. You can find valid values in paragraph 7.4 of this notebook.

Combination of fields work with logical operator AND.
E.g. you will get all records that satisfy all filters.

`text` **AND** `commodity` **AND** `business_category` **AND** `country` **AND** `frequency`

# Imports

# POST data-api/v1/search-count/
Returns count for the given query.

In [3]:
# Build entrypoint url.
count_url = os.path.join(API_URL, "data-api/v1/search-count/")
print("count_url=", count_url)

count_url= https://data.particle.one/data-api/v1/search-count/


In [4]:
# Prepare query.
query = {
    "text": "",
    "commodity": ["Corn"],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [5]:
# Perform query.
response = requests.request("POST", count_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data=", data)

data= {'message': 'OK', 'count': 899}


# POST data-api/v1/search/

- It returns the first chunk of the payload metadata for the given query, where a chunk is 1000 records.
- It also returns `scroll_id` to get the next portion of the data.

In [6]:
search_url = os.path.join(API_URL, "data-api/v1/search/")
print("search_url=", search_url)

search_url= https://data.particle.one/data-api/v1/search/


In [7]:
# Prepare query.
query = {
    "text": "Gas",
    "commodity": [],
    "business_category": "",
    "country": [],
    "frequency": [],
}
payload = json.dumps(query)

In [8]:
# Perform query.
response = requests.request("POST", search_url, headers=HEADERS, data=payload)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

assert "detail" not in data, data

print("total_count=", data["total_count"])

# Saving scroll_id for the next query.
scroll_id = data["scroll_id"]
print("scroll_id=", scroll_id)

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'total_count', 'rows']
total_count= 425473
scroll_id= DXF1ZXJ5QW5kRmV0Y2gBAAAAAABfNdkWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,Accounts Payable of Industrial Enterprises abo...,Crude Oil,4d12ccd2a4a95be4bacbf7d969bb26807191e534,Undefined,China,Annual,100,1949-01-01
1,Accounts Payable of Industrial Enterprises abo...,Crude Oil,326e65235ea7f59719500be55023fd3981982df4,Undefined,China,Annual,100,1949-01-01
2,Accounts Payable of Industrial Enterprises abo...,Crude Oil,b29a00b70540607334fea66f491f7b28e209fc7f,Undefined,China,Annual,100,1949-01-01
3,Accounts Payable of Industrial Enterprises abo...,Natural Gas,4d12ccd2a4a95be4bacbf7d969bb26807191e534,Undefined,China,Annual,100,1949-01-01
4,Accounts Payable of Industrial Enterprises abo...,Natural Gas,326e65235ea7f59719500be55023fd3981982df4,Undefined,China,Annual,100,1949-01-01


# GET data-api/v1/search-scroll/?scroll_id=

In [9]:
# Build entrypoint url.

# We use scroll id from the previous query.
search_scroll_url = os.path.join(
    API_URL, f"data-api/v1/search-scroll/?scroll_id={scroll_id}"
)
print("search_scroll_url=", search_scroll_url)

search_scroll_url= https://data.particle.one/data-api/v1/search-scroll/?scroll_id=DXF1ZXJ5QW5kRmV0Y2gBAAAAAABfNdkWQUFleVN5Ym9RT0daMWhMQU1KNXJ4dw==


In [10]:
# Perform query.

response = requests.request("GET", search_scroll_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))
print("data['rows'][0]=", data["rows"][0])

df = pd.DataFrame.from_records(data["rows"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'scroll_id', 'rows']
data['rows'][0]= {'name': 'Arizona Distillate Fuel Oil – Greater than 15 to 500 ppm Sulfur Stocks at Refineries – Bulk Terminals, and Natural Gas Plants – Thousand Barrels', 'commodity': 'Fuel Oil', 'payload_id': 'b43fe7b80525f1dbe5404c56ceff7919fb188f8c', 'business_category': 'Undefined', 'country': 'United States', 'frequency': 'Monthly', 'unit': 'Thousand Barrels  per Day', 'start_date': None}
df.shape= (1000, 8)
df.head()=


,name,commodity,payload_id,business_category,country,frequency,unit,start_date
0,Arizona Distillate Fuel Oil – Greater than 15 ...,Fuel Oil,b43fe7b80525f1dbe5404c56ceff7919fb188f8c,Undefined,United States,Monthly,Thousand Barrels per Day,None
1,Arizona Distillate Fuel Oil – Greater than 15 ...,Natural Gas,f2dbc21d4e4048f44a030865caee884fbe997d84,Undefined,United States,Annual,Thousand Barrels per Day,None
2,Arizona Distillate Fuel Oil – Greater than 15 ...,Natural Gas,b43fe7b80525f1dbe5404c56ceff7919fb188f8c,Undefined,United States,Monthly,Thousand Barrels per Day,None
3,"Arizona Distillate Fuel Oil, 0 to 15 ppm Sulfu...",Fuel Oil,e38a47c064110a3dfdfeadb867b8d425641cee76,Undefined,United States,Annual,Thousand Barrels per Day,None
4,"Arizona Distillate Fuel Oil, 0 to 15 ppm Sulfu...",Fuel Oil,9abde4c9c16137d0d44b5b26341d07592a0c5843,Undefined,United States,Monthly,Thousand Barrels per Day,None


# GET data-api/v1/payload/?payload_id=
Returns payload for the given `payload_id`

In [11]:
# Build entrypoint url.

# We use one of the `payload_id` from one of the previous queries.
payload_id = "8f26ba4734df3a62352cce9d64987d64da54b400"
payload_url = os.path.join(
    API_URL, f"data-api/v1/payload/?payload_id={payload_id}"
)
print("payload_url=", payload_url)

payload_url= https://data.particle.one/data-api/v1/payload/?payload_id=8f26ba4734df3a62352cce9d64987d64da54b400


In [12]:
# Perform query.
response = requests.request("GET", payload_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["payload_data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'payload_data']
df.shape= (166, 4)
df.head()=


,original_period,original_value,period,value
0,1980,0.0,1980-01-01T00:00:00,0
1,1981,0.0,1981-01-01T00:00:00,0
2,1982,0.0,1982-01-01T00:00:00,0
3,1983,0.0,1983-01-01T00:00:00,0
4,1984,0.0,1984-01-01T00:00:00,0


# Helpers

To search we use several predefined lists of names for each parameter of metadata. Such as:
- `commodity`
- `business-category`
- `country`
- `frequency`

Each parameter has its own set of valid names.

## GET data-api/v1/commodities/

In [13]:
# Build entrypoint url.
commodities_url = os.path.join(API_URL, "data-api/v1/commodities/")
print("commodities_url=", commodities_url)

commodities_url= https://data.particle.one/data-api/v1/commodities/


In [14]:
# Perform query.
response = requests.request("GET", commodities_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (69, 1)
df.head()=


,name
0,Aluminum
1,Benzene
2,Biodiesel
3,Biofuel
4,Butadiene


## GET data-api/v1/business-categories/

In [15]:
# Build entrypoint url.
bc_url = os.path.join(API_URL, "data-api/v1/business-categories/")
print("bc_url=", bc_url)

bc_url= https://data.particle.one/data-api/v1/business-categories/


In [16]:
# Perform query.

response = requests.request("GET", bc_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (4, 1)
df.head()=


,name
0,Downstream
1,Midstream
2,Undefined
3,Upstream


## GET data-api/v1/countries/

In [17]:
# Build entrypoint url.
countries_url = os.path.join(API_URL, "data-api/v1/countries/")
print("countries_url=", countries_url)

countries_url= https://data.particle.one/data-api/v1/countries/


In [18]:
# Perform query.
response = requests.request("GET", countries_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (231, 1)
df.head()=


,name
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


## GET data-api/v1/frequencies/

In [19]:
# Build entrypoint url.
frequencies_url = os.path.join(API_URL, "data-api/v1/frequencies/")
print(frequencies_url)

https://data.particle.one/data-api/v1/frequencies/


In [20]:
# Perform query.
response = requests.request("GET", frequencies_url, headers=HEADERS)
data = json.loads(response.text.encode("utf8"))
print("data.keys()=", list(data.keys()))

df = pd.DataFrame.from_records(data["data"])
print("df.shape=", df.shape)
print("df.head()=")
display(df.head())

data.keys()= ['message', 'data']
df.shape= (6, 1)
df.head()=


,name
0,Daily
1,Weekly
2,Monthly
3,Quarterly
4,Semi-annual
